In [1]:
import torch
import seaborn as sns
import sys
from pathlib import Path
srcdir = Path("../..").resolve()
print(f"Adding {srcdir} to sys.path, this is necessary to import from src")
sys.path.insert(0, str(srcdir))
sys.path

Adding /Users/rgrouls/code/ML22 to sys.path, this is necessary to import from src


['/Users/rgrouls/code/ML22',
 '/Users/rgrouls/code/ML22/notebooks/1_pytroch_intro',
 '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python39.zip',
 '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9',
 '/Users/rgrouls/.pyenv/versions/3.9.16/lib/python3.9/lib-dynload',
 '',
 '/Users/rgrouls/Library/Caches/pypoetry/virtualenvs/deep-learning-HUU8cknU-py3.9/lib/python3.9/site-packages']

In [2]:
from src.models import imagemodels
from src.models import train_model
import gin

In [3]:
gin.parse_config_file("model.gin")

ParsedConfigFileIncludesAndImports(filename='model.gin', imports=['gin.torch.external_configurables'], includes=[])

We will be using `gin-config` to easily keep track of our experiments, and to easily save the different things we did during our experiments.

The `model.gin` file is a simple file that will try to load parameters for funcitons that are already imported. 

So, if you wouldnt have imported train_model, the ginfile would not be able to parse settings for train_model.trainloop and will give an error.

We can print all the settings that are operational with `gin.operative_config_str()` once we have loaded the functions to memory.

In [4]:
from mads_datasets import DatasetFactoryProvider, DatasetType
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
streamers = fashionfactory.create_datastreamer(batchsize=64)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2023-06-06 11:12:51.494 | INFO     | mads_datasets.datasetfactory:download_data:94 - Dataset already exists at /Users/rgrouls/.cache/mads_datasets/fashionmnist
2023-06-06 11:12:51.658 | INFO     | mads_datasets.datasetfactory:download_data:104 - Digest of downloaded /Users/rgrouls/.cache/mads_datasets/fashionmnist/fashionmnist.pt matches expected digest


In [5]:
print(gin.config_str())

import gin.torch.external_configurables

# Parameters for NeuralNetwork:
# ==============================================================================
NeuralNetwork.num_classes = 10
NeuralNetwork.units1 = 512



A big advantage is that we can save this config as a file; that way it is easy to track what you changed during your experiments.

In [6]:
from src.models import metrics
accuracy = metrics.Accuracy()

In [ ]:
import torch.optim as optim
from src.settings import TrainerSettings, ReportTypes
gin.parse_config_file("model.gin")

units = [256, 128, 64]
learning_rate = [0.01, 0.001, 0.0001]
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.GIN],
)

for unit in units:
    for lr in learning_rate:
        gin.bind_parameter("NeuralNetwork.units2", unit)
    
        model = imagemodels.NeuralNetwork()
        trainer = train_model.Trainer(
            model=model, 
            settings=settings, 
            loss_fn=loss_fn,
            optimizer=optim.Adam, 
            traindataloader=trainstreamer, 
            validdataloader=validstreamer, 
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()


Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, navigate to ~/code/ML22 
- activate the python environment for the shell with `poetry shell`. Note how the correct environment is being activated.
- run `tensorboard --logdir=models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`



Experiment with things like:

- changing the amount of units1 and units2 to values between 16 and 1024. Use factors of 2: 16, 32, 64, etc.
- changing the batchsize to values between 4 and 128. Again, use factors of two.
- all your experiments are saved in the `models` directory, with a timestamp. Inside you find a saved_config.gin file, that 
contains all the settings for that experiment. The `events` file is what tensorboard will show.
- plot the result in a heatmap: units vs batchsize.
- changing the learningrate to values between 1e-2 and 1e-5 
- changing the optimizer from SGD to one of the other available algoritms at [torch](https://pytorch.org/docs/stable/optim.html) (scroll down for the algorithms)

A note on train_steps: this is a setting that determines how often you get an update. 
Because our complete dataset is 938 (60000 / 64) batches long, you will need 938 trainstep to cover the complete 60.000 images.

This can actually be a bit confusion, because every value below 938 changes the meaning of `epoch` slightly, because one epoch is no longer
the full dataset, but simply `trainstep` batches. Setting trainsteps to 100 means you need to wait twice as long before you get feedback on the performance,
as compared to trainsteps=50. You will also see that settings trainsteps to 100 improves the learning, but that is simply because the model has seen twice as 
much examples as compared to trainsteps=50.

This implies that it is not usefull to compare trainsteps=50 and trainsteps=100, because setting it to 100 will always be better.
Just pick an amount, and adjust your number of epochs accordingly.